In [3]:
import torch
from accelerate import init_empty_weights
import json
import os

import accelerate
import torch
import transformers
import sys
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

if project_root not in sys.path:
    sys.path.append(project_root)

from model.llama import LlamaForCausalLM

import accelerate
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from transformers import AutoConfig

# ... LlamaForCausalLM, AutoConfig 등 필요한 클래스 import
path = '/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50'

config = AutoConfig.from_pretrained(path)
model_cls = LlamaForCausalLM # 사용자 정의 클래스

model_params = set()
print("--- 모델이 필요로 하는 파라미터 목록 ---")
with init_empty_weights():
    model = model_cls(config)
    for name, _ in model.named_parameters():
        print(name)
        model_params.add(name)
print("------------------------------------")

--- 모델이 필요로 하는 파라미터 목록 ---
model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.q_proj.row_norm
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.k_proj.row_norm
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.v_proj.row_norm
model.layers.0.self_attn.o_proj.weight
model.layers.0.self_attn.o_proj.row_norm
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.gate_proj.row_norm
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.up_proj.row_norm
model.layers.0.mlp.down_proj.weight
model.layers.0.mlp.down_proj.row_norm
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.q_proj.row_norm
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.k_proj.row_norm
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.v_proj.row_norm
model.layers.1.self_attn.o_proj.weight
model.layers.1.self_attn.o_proj.row_norm
mode

In [6]:
import torch
import os
import json

# .bin 파일들이 있는 '디렉토리' 경로를 지정합니다.
path = '/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50'

# 1. 체크포인트 인덱스 파일 경로 지정
index_file = os.path.join(path, "pytorch_model.bin.index.json")

# 2. 인덱스 파일을 열어 가중치 맵(weight_map) 로드
try:
    with open(index_file) as f:
        index = json.load(f)
except FileNotFoundError:
    print(f"오류: 인덱스 파일 '{index_file}'을 찾을 수 없습니다.")
    # 여기서 중단하거나 다른 처리를 할 수 있습니다.

# 모든 분할된 파일들의 이름을 가져옵니다.
shard_files = set(index["weight_map"].values())

all_ckpt_keys = set()
print("--- 모든 분할된 체크포인트에서 가중치 목록 로드 중 ---")

# 3. 모든 분할된 파일을 순회하며 키를 합칩니다.
for shard_file in shard_files:
    shard_path = os.path.join(path, shard_file)
    print(f"'{shard_path}' 파일 로드 중...")
    shard_ckpt = torch.load(shard_path, map_location="cpu")
    all_ckpt_keys.update(shard_ckpt.keys())

print("\n--- 모든 가중치 목록 로드 완료 ---")
print(f"총 {len(shard_files)}개 파일에서 {len(all_ckpt_keys)}개의 고유한 키를 찾았습니다.")
# for key in sorted(list(all_ckpt_keys)):
#     print(key) # 너무 많으면 주석 처리
print("---------------------------------")

--- 모든 분할된 체크포인트에서 가중치 목록 로드 중 ---
'/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50/pytorch_model-00007-of-00007.bin' 파일 로드 중...
'/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50/pytorch_model-00004-of-00007.bin' 파일 로드 중...
'/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50/pytorch_model-00001-of-00007.bin' 파일 로드 중...
'/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50/pytorch_model-00005-of-00007.bin' 파일 로드 중...
'/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50/pytorch_model-00002-of-00007.bin' 파일 로드 중...
'/workspace/Weight_compression/hf_model_comp/comp_qtip/hf/meta-llama--Meta-Llama-3-8B/ql_rnorm_test/lmbda50/pytorch_model-00006-of-00007.bin' 파일 로드 중...
'/workspace/Weight_compression/hf_model_comp/co

In [7]:
print(f"모델 파라미터 개수: {len(model_params)}")
print(f"체크포인트 키 개수: {len(all_ckpt_keys)}")

missing_in_ckpt = model_params - all_ckpt_keys
if missing_in_ckpt:
    print("\n[오류 원인] 모델에는 있지만 체크포인트에는 없는 파라미터:")
    for name in sorted(list(missing_in_ckpt)):
        print(name)

extra_in_ckpt = all_ckpt_keys - model_params
if extra_in_ckpt:
    print("\n참고: 체크포인트에는 있지만 모델에는 없는 가중치:")
    for name in sorted(list(extra_in_ckpt)):
        print(name)

모델 파라미터 개수: 515
체크포인트 키 개수: 515
